In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [3]:
!git clone https://github.com/tenoke/gpt-2

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [0]:
os.chdir('gpt-2')

In [5]:
!pip3 install -r requirements.txt

In [0]:
!mkdir .kaggle

In [9]:
ls

CONTRIBUTORS.md  Dockerfile.gpu     gpt-2-samples/  requirements.txt  train.py*
DEVELOPERS.md    download_model.py  LICENSE         src/
Dockerfile.cpu   encode.py*         README.md       train-horovod.py


In [0]:
token = {"username":"xianshou","key":"9df2f9227d52fa331b2ea240826b976f"}
with open('/content/gpt-2/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/gpt-2/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [12]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [14]:
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus -p /content/gpt-2 --force

 99% 788M/799M [00:09<00:00, 114MB/s] 
100% 799M/799M [00:09<00:00, 90.8MB/s]


In [15]:
!unzip ubuntu-dialogue-corpus.zip

Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


In [0]:
#By and large if we want to use all datasets - uncomment as necessary.
#For this exercise I have only used dialogueText_301.csv

#dialogue_1 = 'Ubuntu-dialogue-corpus/dialogueText.csv'
#dialogue_2 = 'Ubuntu-dialogue-corpus/dialogueText_196.csv'
dialogue_3 = 'Ubuntu-dialogue-corpus/dialogueText_301.csv'

#d1_df = pd.read_csv(dialogue_1, parse_dates=['date'], chunksize=1200000)
#d2_df = pd.read_csv(dialogue_2, parse_dates=['date'], chunksize=1200000)
d3_df = pd.read_csv(dialogue_3, parse_dates=['date'], chunksize=1200000)


In [17]:
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 793kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 57.8Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 801kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:22, 62.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.71Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 60.1Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 56.4Mit/s]                                                       


In [0]:
!mkdir ubuntu-data ubuntu-npz

In [0]:
#function creates a processed version of the csv
#extracting the content we need: date, sender, message
def clean(dataset, file_name):
  i = 1
  for data in dataset:    
    text_corpus = ''
    current = None
    for msg in data.itertuples():
      if msg.dialogueID != current:
        current = msg.dialogueID
        text_corpus += '\n\n'
      try: 
        text_corpus += f"({msg.date}) {msg._4}: {msg.text}\n"
      except KeyError:
        pass
    
    
    with open(f'ubuntu-data/{file_name}_{i}.txt', 'w') as f:
      f.write(text_corpus)  
    del(text_corpus)
    i += 1
    
  
  

In [0]:
#We only read the d3_df so we are only cleaning that file
#clean(d1_df, 'cleaned_d1_df')
#(d2_df, 'cleaned_d2_df')
clean(d3_df, 'cleaned_d3_df')

In [22]:
#Encoding processed files generated from the csv as necessary
#to find this file I used the 'ls' command
#after changing to the appropriate directory
 
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d1_df_1.txt ubuntu-npz/cleaned_d1_df_1.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_1.txt ubuntu-npz/cleaned_d2_df_1.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_2.txt ubuntu-npz/cleaned_d2_df_2.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_3.txt ubuntu-npz/cleaned_d2_df_3.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_4.txt ubuntu-npz/cleaned_d2_df_4.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_5.txt ubuntu-npz/cleaned_d2_df_5.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_6.txt ubuntu-npz/cleaned_d2_df_6.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_7.txt ubuntu-npz/cleaned_d2_df_7.txt.npz --model_name 345M
#!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_8.txt ubuntu-npz/cleaned_d2_df_8.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_1.txt ubuntu-npz/cleaned_d3_df_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_2.txt ubuntu-npz/cleaned_d3_df_2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_3.txt ubuntu-npz/cleaned_d3_df_3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_4.txt ubuntu-npz/cleaned_d3_df_4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_5.txt ubuntu-npz/cleaned_d3_df_5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_6.txt ubuntu-npz/cleaned_d3_df_6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_7.txt ubuntu-npz/cleaned_d3_df_7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_8.txt ubuntu-npz/cleaned_d3_df_8.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_9.txt ubuntu-npz/cleaned_d3_df_9.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_10.txt ubuntu-npz/cleaned_d3_df_10.txt.npz --model_name 345M

Reading files
100% 1/1 [03:31<00:00, 211.21s/it]
Writing ubuntu-npz/cleaned_d3_df_1.txt.npz
Reading files
100% 1/1 [03:39<00:00, 219.31s/it]
Writing ubuntu-npz/cleaned_d3_df_2.txt.npz
Reading files
100% 1/1 [03:32<00:00, 212.86s/it]
Writing ubuntu-npz/cleaned_d3_df_3.txt.npz
Reading files
100% 1/1 [03:32<00:00, 212.35s/it]
Writing ubuntu-npz/cleaned_d3_df_4.txt.npz
Reading files
100% 1/1 [03:41<00:00, 221.29s/it]
Writing ubuntu-npz/cleaned_d3_df_5.txt.npz
Reading files
100% 1/1 [03:54<00:00, 234.07s/it]
Writing ubuntu-npz/cleaned_d3_df_6.txt.npz
Reading files
100% 1/1 [03:55<00:00, 235.51s/it]
Writing ubuntu-npz/cleaned_d3_df_7.txt.npz
Reading files
100% 1/1 [03:37<00:00, 217.52s/it]
Writing ubuntu-npz/cleaned_d3_df_8.txt.npz
Reading files
100% 1/1 [03:40<00:00, 220.32s/it]
Writing ubuntu-npz/cleaned_d3_df_9.txt.npz
Reading files
100% 1/1 [04:41<00:00, 281.58s/it]
Writing ubuntu-npz/cleaned_d3_df_10.txt.npz


In [0]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --stop_after 1501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 12:03:37.208674: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 12:03:37.210462: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x21d9800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 12:03:37.210504: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [0]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --learning_rate 0.00001 --stop_after 3501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 04:34:18.584810: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 04:34:18.586377: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x13f1800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 04:34:18.586415: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [0]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --learning_rate 0.000001 --stop_after 5001 --model_name 345M

In [0]:
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.95 --model_name 345M